## Import

In [ ]:
import ee
import pandas as pd
import os
import time
import shutil

from calendar import monthrange

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=A6WcZPigwht0Aj435kje5teQCGTPAJNsWBLqVGM7Me0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g45sIfzqpamTIgX_TMdRE485VSdOPUNv39zUtfGiw1Gup77_8V7qfE

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Load cities coordinates

In [ ]:
df_coord = pd.read_csv(os.path.join('drive','MyDrive','MLBDproject','Data','european_cities.csv'))

In [ ]:
df_coord = df_coord.iloc[:55] # we get the 55 first cities

In [ ]:
df_coord

,Country,City,Latitude,Longitude
0,France,Paris,48.8566,2.3522
1,France,Nice,43.7034,7.2663
2,France,Toulouse,43.6045,1.4440
3,France,Marseille,43.2964,5.3700
4,France,Rennes,48.1147,-1.6794
5,France,Grenoble,45.1715,5.7224
6,France,Nantes,47.2181,-1.5528
7,France,Montpellier,43.6119,3.8772
8,France,Lyon,45.7600,4.8400
9,Germany,Berlin,52.5167,13.3833


## Get the mean value for a given region in Google Earth Engine

In [ ]:
VIIRSCollection = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').select('avg_rad')
VIIRSParams = {'bands':['avg_rad']}

In [ ]:
mean_values = {}

In [ ]:
delta_lng_area = 0.32135009765
delta_lat_area = 0.32135009765

for i in range(len(df_coord)):
  lat = df_coord.iloc[i]['Latitude']
  lng = df_coord.iloc[i]['Longitude']
  city = df_coord.iloc[i]['City']

  geometry = ee.Geometry.Polygon(
  [[[lng-delta_lng_area, lat+delta_lat_area],
  [lng-delta_lng_area, lat-delta_lat_area],
  [lng+delta_lng_area, lat-delta_lat_area],
  [lng+delta_lng_area, lat+delta_lat_area]]], None, False);

  mean_values[city] = []

  for year in range(2014,2021):
    for month in range(1,13):

      img = VIIRSCollection.filterDate(str(year) + '-' + str(month) + '-01', str(year) + '-' + str(month) + '-' + str(monthrange(year, month)[1])).median()

      mean_val = img.reduceRegion(
          geometry = geometry,
          reducer= ee.Reducer.mean(),
          scale= 500)
      mean_values[city].append(mean_val.getInfo()['avg_rad'])



  print(f"Ready for {city}")

Ready for Paris
Ready for Nice
Ready for Toulouse
Ready for Marseille
Ready for Rennes
Ready for Grenoble
Ready for Nantes
Ready for Montpellier
Ready for Lyon
Ready for Berlin
Ready for Hamburg
Ready for Munich
Ready for Cologne
Ready for Frankfurt
Ready for Bremen
Ready for Stuttgart
Ready for Düsseldorf
Ready for Dortmund
Ready for Essen
Ready for London
Ready for Birmingham
Ready for Manchester
Ready for Leeds
Ready for Newcastle
Ready for Birstall
Ready for Glasgow
Ready for Liverpool
Ready for Portsmouth
Ready for Southampton
Ready for Rome
Ready for Milan
Ready for Naples
Ready for Turin
Ready for Palermo
Ready for Genoa
Ready for Bologna
Ready for Florence
Ready for Bari
Ready for Catania
Ready for Barcelona
Ready for Madrid
Ready for Sevilla
Ready for Valencia
Ready for Zaragoza
Ready for Lisbon
Ready for Vila Nova de Gaia
Ready for Porto
Ready for Braga
Ready for Matosinhos
Ready for Zürich
Ready for Geneva
Ready for Basel
Ready for Lausanne
Ready for Bern
Ready for Neuchâtel

## Compute datetime index

In [ ]:
import datetime
def compute_months(first_date, second_date):
    year1, month1, year2, month2 = map(
        int, 
        (first_date[:4], first_date[5:7], second_date[:4], second_date[5:7])
    )

    return [
        '{:0>4}-{:0>2}'.format(year, month)
        for year in range(year1, year2 + 1)
        for month in range(month1 if year == year1 else 1, month2 + 1 if year == year2 else 13)
    ]
date_index = [datetime.date.fromisoformat(i + '-01') for i in compute_months("2014-01", "2020-12")]

## Save dataframe on the disk



In [ ]:
df_mean = pd.DataFrame().from_dict(mean_values)
df_mean['date'] = date_index
df_mean = df_mean.set_index(df_mean['date'], drop=True)
del df_mean['date']
df_mean = df_mean.T
df_mean.to_csv(os.path.join('drive','MyDrive','MLBDproject','Data','european_cities_light.csv'))

In [ ]:
df_mean

date,2014-01-01,2014-02-01,2014-03-01,2014-04-01,2014-05-01,2014-06-01,2014-07-01,2014-08-01,2014-09-01,2014-10-01,2014-11-01,2014-12-01,2015-01-01,2015-02-01,2015-03-01,2015-04-01,2015-05-01,2015-06-01,2015-07-01,2015-08-01,2015-09-01,2015-10-01,2015-11-01,2015-12-01,2016-01-01,2016-02-01,2016-03-01,2016-04-01,2016-05-01,2016-06-01,2016-07-01,2016-08-01,2016-09-01,2016-10-01,2016-11-01,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,...,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01
Paris,23.002404,24.346827,24.338486,24.184378,21.688179,24.448322,23.637670,20.795273,23.898814,21.348290,18.925169,16.904693,21.393762,24.067471,25.048185,26.284566,23.811161,23.951375,21.695851,22.746927,23.831317,22.151410,20.232939,23.001406,22.073825,18.461824,24.531599,23.912475,24.145807,21.628234,21.938150,23.044015,22.656185,23.246499,23.045213,21.782955,21.263120,19.836840,24.605254,25.641184,...,22.228051,21.426463,19.831250,17.119539,16.098812,26.367485,23.876168,25.249155,24.492906,22.679124,22.306680,23.314512,23.872423,24.461883,23.164000,18.982566,19.878132,22.162996,25.244751,24.730430,23.028351,22.891253,23.308276,23.275769,22.501918,13.023005,16.842984,21.719124,20.031888,24.345317,24.728727,23.585980,22.833114,22.576400,21.454461,21.062295,22.585774,19.348862,17.226775,13.678557
Nice,3.363471,3.442395,3.572344,3.450301,3.658497,3.586682,3.465273,3.679672,3.697288,3.502259,3.275873,3.897228,3.737722,3.532499,3.382477,3.352624,3.412183,3.396009,3.206505,3.438708,3.348546,3.356548,3.414846,3.334268,3.664693,3.456115,3.588502,3.535892,3.595122,3.273324,3.333105,3.372738,3.374104,3.452917,3.545776,3.701517,3.612269,3.652119,3.764965,3.716753,...,3.527187,3.704021,3.681178,3.665920,3.729302,3.463683,3.649015,3.520742,3.453347,3.473165,3.310391,3.623554,3.421766,3.450795,3.189871,3.461195,3.623000,3.568817,3.606043,3.554658,3.380149,3.448729,3.236167,3.359780,3.401590,3.405616,3.531982,3.491517,3.372263,3.524579,3.356697,3.222440,3.279513,3.503150,3.040913,3.306960,3.146317,3.171306,3.491335,3.214498
Toulouse,4.777807,5.027478,4.550872,5.379510,5.071400,5.530831,5.126738,5.415983,5.621796,5.611113,4.868033,4.682496,4.459988,5.481303,4.777516,5.770592,4.766844,4.994827,5.019634,4.932079,4.902588,4.995869,4.792578,4.910815,4.733455,4.932711,4.511391,4.580343,4.996421,4.794584,4.924771,4.747092,4.969972,5.020869,4.263931,4.540260,4.690059,5.003515,4.868497,5.221795,...,4.559628,4.734029,4.903725,3.807077,3.564921,3.906629,4.534160,4.866818,4.398246,4.396805,4.397658,4.710411,4.356869,4.080970,4.443659,3.726280,3.609726,4.615701,4.716353,4.573741,4.661751,4.586935,4.300631,4.472789,4.272492,4.263390,3.236323,3.838635,3.869625,4.323834,4.444210,4.418120,4.526456,4.576661,4.321123,4.149306,4.072029,3.868599,4.403189,3.195357
Marseille,5.227348,6.005932,6.101828,5.819513,5.943394,5.788195,5.846998,5.710302,5.990193,5.740263,5.143129,6.199648,6.224660,5.926423,6.079247,5.892932,6.009496,5.564440,5.534647,5.338763,5.717325,5.728126,5.592378,5.422315,5.955174,5.742369,5.703941,5.730170,6.955947,5.684442,5.604915,5.727759,5.579403,5.773537,5.451330,6.030942,5.803424,5.865186,5.925928,6.083406,...,5.613406,5.994043,5.971078,5.726080,5.764787,5.957548,6.708877,5.877643,5.520136,5.900590,5.808570,5.593180,5.532167,5.678826,5.048360,5.508136,5.722295,5.726632,6.116732,5.527920,5.859738,7.122944,5.431863,5.436334,5.485070,5.606645,5.786472,5.646440,5.335925,5.745050,5.813838,5.234923,5.623868,5.914786,5.634603,5.491193,5.052157,5.509267,5.696107,5.358745
Rennes,4.203696,2.985455,1.347323,1.332537,1.288814,1.734468,1.587255,1.397252,1.548241,1.340778,3.618831